<a href="https://colab.research.google.com/github/rexbrandy/Neural_Networks/blob/main/FashionMNIST_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fashion MNIST Classifier

### Import everything and define train and test sets/loaders

In [10]:
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as datasets


assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Declare the transform
transform = transforms.Compose(
    [transforms.ToTensor(), # Transform image to tensor
     transforms.Normalize((0.5,),(0.5,))] # Normalize data
)

# Declare training and testing sets
train_set = datasets.FashionMNIST('data',
  download=True,
  train=True,
  transform=transform
)
test_set = datasets.FashionMNIST('data',
  download=True,
  train=False,
  transform=transform
)

# Declare dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64,
                                           shuffle=True, num_workers=1)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=64,
                                           shuffle=True, num_workers=1)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img=img.mean(dim=0)
    img = img / 2 + 0.5
    np_img = img.numpy()
    if one_channel:
        plt.imshow(np_img, cmap='Greys')
    else:
        plt.imshow(np.transpose(np_img, (1, 2, 0)))

  

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Define our classifier

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256) # Input layer should be 784 because images are 28x28 - one for every feature (pixel)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10) # we have 10 classes so output should be one of those

    def forward(self, x):
        # make sure input is flat
        x = x.view(x.shape[0], -1)

        # for every layer we should activate output with ReLU
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        

### Define loss function and optimizer 

In [12]:
import torch.optim as optim

# Create model instance
model = Classifier()

# Move to GPU
model.cuda()

# Define metric for loss function
criterion = nn.NLLLoss()

# Select optimizer and pass learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Test gradient computation

In [15]:
images, labels = next(iter(train_loader))
log_ps = model(images.cuda())
loss = criterion(log_ps, labels.cuda())

loss.backward()

print(model.fc2.weight.grad)

tensor([[-2.8631e-05, -6.7111e-04, -5.9635e-04,  ...,  4.8667e-07,
          1.1473e-06, -5.3440e-04],
        [-6.1012e-06, -9.5925e-05, -5.7284e-04,  ...,  5.2530e-05,
         -4.4380e-04, -1.6100e-04],
        [-8.0933e-04,  7.2123e-05,  5.6577e-04,  ..., -2.0457e-04,
         -1.0760e-04,  8.5038e-05],
        ...,
        [ 1.1763e-03,  6.2503e-04,  8.2013e-04,  ...,  1.9332e-05,
          3.1753e-04,  2.2473e-04],
        [-1.6492e-03,  3.6530e-05, -4.6334e-04,  ..., -8.4973e-05,
         -5.2131e-04,  3.4158e-04],
        [ 4.0237e-05, -3.3123e-04,  5.6661e-04,  ...,  0.0000e+00,
         -2.2045e-04, -5.9264e-05]], device='cuda:0')
